In [79]:
import cv2
import numpy as np
import matplotlib as plt
import random

Define Functions

In [80]:
def isSegment(p1,p2, bin_im, ksize, sample_num, pvote, cvote):
    x1, y1 = p1
    x2, y2 = p2
    
    x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)
    
    x_disp = x2- x1
    y_disp = y2- y1
    
    x_step = x_disp // (sample_num + 1)
    y_step = y_disp // (sample_num + 1)
    
    vote = 0

    for mult in range(1, sample_num + 1):  
        xc, yc = x1 + mult*x_step, y1 + mult*y_step
        view = bin_im[xc - ksize: xc + ksize, yc - ksize: yc + ksize]
        
        if np.sum(view) >= pvote:
            vote += 1 

    return vote >= cvote

In [81]:
def getCorners(img):   
    gray = np.float32(img)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
    dst = np.uint8(dst)
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)
    
    return corners

In [99]:
def getSegments(img, corners, ksize = 2, sample_num = 15, pvote = 3, cvote = 5):
    segments = []

    for i in range(len(corners)):
        for j in range(i + 1, len(corners)):
            c1, c2 = corners[i], corners[j]
            if isSegment(c1, c2, img, ksize = 2, sample_num = 15, pvote = 3, cvote = 5):
                segments.append((tuple(c1), tuple(c2)))
            
    return segments


## Test Functions

##### hyperparameters

In [107]:
PVOTE = 3
CVOTE = 5
KSIZE = 3
SAMPLE_NUM = 15

img_directory = "images/"

##### test script

In [110]:
im = cv2.imread(img_directory + "frog.png")
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# "Preps Image to be displayed" - Lillian
im_display = np.zeros(im.shape, dtype = np.uint8)
im_display.fill(255)

corners = getCorners(im > 250)
segments = getSegments(im, corners, ksize = KSIZE, pvote = PVOTE, cvote = CVOTE)


print("DETECTED SEGMENTS")
for i in segments:
    cv2.line(im_display, i[0], i[1], (0,0,255),1) 

print("First segment: ", segments[1])
cv2.line(im_display, segments[1][0], segments[1][1], (0,0,255), 3)

    # print("("+ str(i) +","+str(segments.get(i))+")")

# cv2.circle(im_display, (int(1.6140463), int(199.0812)), 2, (0,255,0), thickness=1, lineType=8, shift=0) 
# cv2.circle(im_display, (int(196.46725), int(197.349)), 2, (0,255,0), thickness=1, lineType=8, shift=0) 

#test = isSegment((1.6140463, 199.0812),(196.46725, 197.349 ), img)
#print test

cv2.imwrite(img_directory + "cornersdisplay" + str(KSIZE) +  str(PVOTE) + str(CVOTE) + str(SAMPLE_NUM) + ".png", im_display)
cv2.imwrite(img_directory + "frogcorners.png", im)


DETECTED SEGMENTS
First segment:  ((195.16862, 195.16862), (199.0733, 1.5104129))


True